In [3]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WG=pd.read_csv('/content/BRCA_FR_AF_superFILTRO_sample_inputmatrix.txt',sep='\t')

In [4]:
BRCA_FR_SUPERFILTRATO_WG

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,202468579,202468579,C,A,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
418581,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
418582,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
418583,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
418584,BRCA-FR,SA595120,SA595120,GRCh37,SNP,6,42376519,42376519,C,T,SOMATIC


In [5]:
ESONI=pd.read_csv('/content/drive/MyDrive/ESONI.tsv',sep='\t')

In [6]:
BRCA_FR_SUPERFILTRATO_WG['chrom']='chr'+ BRCA_FR_SUPERFILTRATO_WG['chrom'].astype(str)

In [7]:
ESONI['chrom']=ESONI['chrom'].astype(str)

In [8]:
ESONI['exonStarts']=ESONI['exonStarts'].astype(int)

In [9]:
ESONI['exonEnds']=ESONI['exonEnds'].astype(int)

In [10]:
BRCA_FR_SUPERFILTRATO_WG['pos_start']=BRCA_FR_SUPERFILTRATO_WG['pos_start'].astype(int)
BRCA_FR_SUPERFILTRATO_WG['pos_end']=BRCA_FR_SUPERFILTRATO_WG['pos_end'].astype(int)

In [11]:
n=int(0.2*(ESONI.shape[0]))

In [12]:
ESONI=ESONI.sample(n)

In [13]:
ESONI

,chrom,exonStarts,exonEnds,name2
69289,chr4,159122563,159124029,GASK1B-AS1
150146,chr10,123629520,123629553,ATE1
177380,chr12,114823280,114823372,TBX5
232687,chr17,42200621,42201009,HDAC5
87010,chr6,44409626,44409650,POLR1C
...,...,...,...,...
244129,chr19,39592105,39592177,ACP7
283259,chr6_ssto_hap7,2880833,2881002,LTB
60173,chr3,101578161,101579866,NFKBIZ
264134,chr21,25801053,25801218,LINC01684


In [14]:
import pandas as pd
import numpy as np
from multiprocessing import Pool

# Definizione della funzione per il confronto dei blocchi
def compare_blocks(block_BRCA, block_ESONI):
    # Resto del codice rimane invariato
    merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
    filtered_block = merged_block[
        (merged_block['pos_start'] >= merged_block['exonStarts']) &
        (merged_block['pos_end'] <= merged_block['exonEnds'])
    ]
    return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start',
       'pos_end', 'ref', 'alt', 'Type']]

# Dimensione dei blocchi
block_size = 1000

# Calcolo del numero di blocchi per ciascun DataFrame
num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

# Creazione della lista di indici per i blocchi
block_indices = []
for i in range(num_blocks_BRCA):
    start_idx_BRCA = i * block_size
    end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

    for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

# Funzione wrapper per la chiamata a compare_blocks con gli indici
def compare_blocks_wrapper(indices):
    block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
    block_ESONI = ESONI.iloc[indices[2]:indices[3]]

    return compare_blocks(block_BRCA, block_ESONI)

# Utilizzo di multiprocessing per eseguire operazioni parallele
with Pool() as pool:
    results = pool.map(compare_blocks_wrapper, block_indices)

# Combinazione dei risultati in un unico DataFrame
final_result = pd.concat(results, ignore_index=True)

In [15]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr19,12984581,12984581,C,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr19,12984581,12984581,C,T,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr19,12984581,12984581,C,T,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr19,12984581,12984581,C,T,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,chr19,12984581,12984581,C,T,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
16631,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr17,37881000,37881000,G,T,SOMATIC
16632,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr14,35181006,35181006,C,T,SOMATIC
16633,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr14,35181006,35181006,C,T,SOMATIC
16634,BRCA-FR,SA595120,SA595120,GRCh37,SNP,chr14,35181006,35181006,C,T,SOMATIC


In [16]:
for i in range(final_result.shape[0]):
  final_result.loc[i,'chrom']= final_result.loc[i,'chrom'][3:]

In [17]:
final_result

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,19,12984581,12984581,C,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,19,12984581,12984581,C,T,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,19,12984581,12984581,C,T,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,19,12984581,12984581,C,T,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,19,12984581,12984581,C,T,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
16631,BRCA-FR,SA595120,SA595120,GRCh37,SNP,17,37881000,37881000,G,T,SOMATIC
16632,BRCA-FR,SA595120,SA595120,GRCh37,SNP,14,35181006,35181006,C,T,SOMATIC
16633,BRCA-FR,SA595120,SA595120,GRCh37,SNP,14,35181006,35181006,C,T,SOMATIC
16634,BRCA-FR,SA595120,SA595120,GRCh37,SNP,14,35181006,35181006,C,T,SOMATIC


In [18]:
# Salva il DataFrame come file di testo separato da virgole (CSV)
final_result.to_csv('brca_fr_target_superfiltrato.txt', sep='\t', index=False)  # Separa le colonne con il carattere di tabulazione ('\t')